# <center >TAPE

- `transformer` (pretrained available)
- `resnet`
- `lstm`
- `unirep` (pretrained available)
- `onehot` (no pretraining required)
- `trrosetta` (pretrained available)

In [1]:
import numpy as np
import pandas as pd
import altair      as alt

import torch
from tqdm import tqdm_notebook as tqdm
from tape import TAPETokenizer, ProteinBertModel # ProteinLSTMModel

In [2]:
#df = pandas.read_csv("../data/macros.csv", index_col="Uniprot Code")
df = pd.read_excel("../data/macros&covid_with_domains.xlsx", index_col="Uniprot Code")
df.head()

,Tipo de Macro,Secuencia,uniprot,Unnamed: 4,domain,accession,hit_start,hit_end,domain.1,accession.1,...,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159
Uniprot Code,,,,,,,,,,,,,,,,,,,,,
O28751,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,O28751,NaN,MACRO,PS51154,1,192,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D3RWS7,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...,D3RWS7,NaN,MACRO,PS51154,3,193,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D2RH24,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...,D2RH24,NaN,MACRO,PS51154,1,191,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0A0F7ICE9,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...,A0A0F7ICE9,NaN,MACRO,PS51154,1,194,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0A075LQ95,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...,A0A075LQ95,NaN,MACRO,PS51154,1,190,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# BERT

In [3]:
# Pfam Family: Hexapep, Clan: CL0536
sequence  = 'GCTVEDRCLIGMGAILLNGCVIGSGSLVAAGALITQ'
tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model
token_ids = torch.tensor([tokenizer.encode(sequence)]).cuda()
token_ids

tensor([[ 2, 11,  7, 23, 25,  9,  8, 21,  7, 15, 13, 11, 16, 11,  5, 13, 15, 15,
         17, 11,  7, 25, 13, 11, 22, 11, 22, 15, 25,  5,  5, 11,  5, 15, 13, 23,
         20,  3]], device='cuda:0')

In [4]:
bert = ProteinBertModel.from_pretrained('bert-base').cuda().eval()

# NOTE: pooled_output is *not* trained for the transformer, do not use
# w/o fine-tuning. A better option for now is to simply take a mean of
# the sequence output
output = bert(token_ids)
sequence_output = output[0]
pooled_output = output[1]

sequence_output.shape, pooled_output.shape

(torch.Size([1, 38, 768]), torch.Size([1, 768]))

# Inferirir Macros

In [5]:
seqs        = df["Secuencia"].values
seqs_tokens =  [torch.tensor([tokenizer.encode(seq)]).cuda() for seq in seqs]

In [8]:
outputs = []

for seq_tokens in tqdm(seqs_tokens):
    with torch.no_grad():
        output = bert(seq_tokens)[1][0].cpu().numpy()
    
    print(output.shape)
    outputs.append(output)

<ipython-input-8-48d4cd07571d>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for seq_tokens in tqdm(seqs_tokens):


(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)
(768,)

In [9]:
outputs_np = np.array(outputs)
outputs_np.shape

(176, 768)

In [10]:
np.save("macros_bert.npy", outputs_np)